<a href="https://colab.research.google.com/github/Siddharthg97/Covid-19-dashboard/blob/main/Name_%26_DOB_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
import re

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import pandas as pd
import numpy as np
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import ast

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)

cuda


In [ ]:
# l_samp=pd.read_csv("/content/complete_prompt.csv")
l_samp=pd.read_csv("/content/sample_columns_prompt_with_dob_name_data.csv")

In [ ]:
l_samp.shape

(64327, 4)

In [ ]:
l_samp[0:5]

,Unnamed: 0,prompt,comments,prompt_&_comment
0,0,The customer and watch person date of birth is...,Mismatch Full Legal Name_Name Does Not Sound S...,The customer and watch person date of birth is...
1,1,The customer and watch person date of birth is...,Mismatch Full Legal Name_Name Does Not Sound S...,The customer and watch person date of birth is...
2,2,The customer and watch person date of birth is...,Mismatch Full Legal Name_Name Does Not Sound S...,The customer and watch person date of birth is...
3,3,The customer and watch person date of birth is...,Mismatch Full Legal Name_Name Does Not Sound S...,The customer and watch person date of birth is...
4,4,The customer and watch person date of birth is...,Mismatch Full Legal Name_Name Does Not Sound S...,The customer and watch person date of birth is...


In [ ]:
# l_samp["prompt"]=l_samp["prompt"].apply(lambda x: x+"." if "Hit." not in x else x )

In [ ]:
# l_samp["prompt_&_comment"]=l_samp[["prompt","comments"]].apply(lambda x: x[0]+x[1],axis=1)

In [ ]:
#Create a very small test set to compare generated text with the reality
l_test= l_samp.sample(100)
l_train = l_samp.loc[~l_samp.index.isin(l_test.index)]

#Reset the indexes
l_test = l_test.reset_index()


In [ ]:
l_train.shape,l_samp.shape,l_test.shape

((64227, 4), (64327, 4), (100, 5))

In [ ]:
l_test["prompt_&_comment"][0]

"The customer date of birth  and watch person  year of birth is 1985/08/02 and ['1994', '1995', '1996', '1997', '1998'] respectively,with their date of birth similarity score as 0.0.Their names are MOHD  YUSOFF BIN AWANG and  YUSOFF MOHAMAD respectively, with  their string based name matching scores as 1.645941716255095e-10, 100.0 and sound based name matching score as 0.5882352941176471.Thus, customer and watchlist are a mis-match.Mismatch Year of Birth (YOB)"

In [ ]:
l_test[["prompt_&_comment","prompt","comments"]].head()

,prompt_&_comment,prompt,comments
0,The customer date of birth and watch person ...,The customer date of birth and watch person ...,Mismatch Year of Birth (YOB)
1,The customer and watch person date of birth is...,The customer and watch person date of birth is...,Mismatch Date of Birth (DOB)
2,The customer date of birth and watch person ...,The customer date of birth and watch person ...,Mismatch Year of Birth (YOB)
3,The customer date of birth and watch person ...,The customer date of birth and watch person ...,Mismatch Year of Birth (YOB)
4,The customer and watch person date of birth is...,The customer and watch person date of birth is...,Mismatch Full Legal Name/Date of Birth (DOB)/N...


In [ ]:
l_train.reset_index(inplace=True)

In [ ]:
package="gpt2"

In [ ]:
class data_train(Dataset):
    def __init__(self, control_code, truncate=False, gpt2_type=package, max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.comments = []

        for row in l_train['prompt_&_comment']:

            self.comments.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>",truncation=True,max_length=max_length
            )).to(device))
        if truncate:
            self.comments = self.comments[:20000]
        self.comments_count = len(self.comments)

    def __len__(self):
        return self.comments_count

    def __getitem__(self, item):
        return self.comments[item]



In [ ]:
dataset = data_train(l_train['prompt_&_comment'], truncate=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Training epoch is 1

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=1, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir="/content/sample_data", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=True,
):
    acc_steps = 100
    model = model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
# output_dir="/content/sample_data"
# output_prefix="wreckgar"
# epoch=1
# torch.save(
#                 model.state_dict(),
#                 os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
#             )


In [ ]:
model = train(dataset, model, tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


20000it [26:28, 12.59it/s]


In [ ]:
# Torch model saving
torch.save({'epoch': 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': .state_dict(),
            'loss': LOSS},
	    'save/to/path/model.pth')


SyntaxError: invalid syntax (<ipython-input-22-2577859e8983>, line 4)

In [ ]:
# Text generation

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=50, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_comments = []
  for i in range(len(test_data)):
    x = generate(model.to("cpu"), tokenizer, test_data["prompt"][i], entry_count=1)
    generated_comments.append(x)

  return generated_comments



In [ ]:
l_test["comments"].value_counts()

Mismatch Year of Birth (YOB)                                                                                                                                                                                                  40
Mismatch Full Legal Name_Name Does Not Sound Similar                                                                                                                                                                          30
Mismatch Date of Birth (DOB)                                                                                                                                                                                                  13
Mismatch Full Legal Name_ Name Does Not Sound Similar - Rationale: Though the name has  close match e.g. only one/ two different character(s), it does not sound similar and highly unlikely that this is the same person.     4
Mismatch Date of Birth (DOB)/NRIC number - Both Customer and Watchperson have different NRIC number 

In [ ]:
#Run the functions to generate the
l_test["comments_gen"]=text_generation(l_test)

100%|██████████| 1/1 [00:28<00:00, 28.86s/it]


In [ ]:
l_test["comments_gen"].apply(lambda x: x.split("match.",1)[1])[0]

NameError: name 'l_test' is not defined

In [ ]:
l_test["Only_comments_gen"]=l_test["comments_gen"].apply(lambda x: x.split("match.",1)[1].rstrip('<|endoftext|>'))

In [ ]:
# l_test["comments_gen"]
# l_test["Only_comments_gen"]=l_test["comments_gen"].apply(lambda x: x.split("match.",1)[1].strip("<|endoftext|>"))

In [ ]:
# l_test["comments_gen"]=l_test["comments_gen"].apply(lambda x: x[0])
# l_test["Only_comments_gen"]=l_test["comments_gen"].apply(lambda x: x.split("match.",1)[1].strip("<|endoftext|>"))

In [ ]:
# patt=re.compile("match.")
# l_train_TH=l_samp[l_samp["prompt"].str.contains(patt)].sample(10)
# l_train_TH.reset_index(inplace=True)

In [ ]:
# l_train_TH["comments_gen_TH"]=text_generation(l_train_TH)

100%|██████████| 1/1 [00:00<00:00,  5.24it/s]


['For customer and watch person date of birth are 1983/11/05 and 1983/11/05 and score is 1.0 with resolution True Hit.<|endoftext|>']


100%|██████████| 1/1 [00:00<00:00,  5.95it/s]


['For customer and watch person date of birth are 1976/11/09 and 1976/11/09 and score is 1.0 with resolution True Hit.<|endoftext|>']


100%|██████████| 1/1 [00:00<00:00,  6.01it/s]


...

['For customer and watch person date of birth are 1980/11/30 and 1980/11/30 and score is 1.0 with resolution True Hit.<|endoftext|>']


100%|██████████| 1/1 [00:04<00:00,  4.07s/it]


['For customer and watch person date of birth are 1986/12/23 and 1986/12/23 and score is 1.0 with resolution True Hit.\n\nAll images and text © 1998-2016 Tate and Flair Images Ltd.<|endoftext|>']


100%|██████████| 1/1 [00:00<00:00,  4.87it/s]

['For customer and watch person date of birth are 1969/08/23 and 1969/08/23 and score is 1.0 with resolution True Hit.<|endoftext|>']


In [ ]:
l_test.to_csv("/content/sample_data/comments_gen_with_dob_name.csv")

NameError: name 'l_test' is not defined

In [ ]:
The end

In [ ]:
l_test["comments_gen"]=l_test["comments_gen"].apply(lambda x: x[0])
generated_comments=l_test["comments_gen"]

In [ ]:
# #Loop to keep only generated text and add it as a new column in the dataframe
# my_generations=[]

# for i in range(len(generated_comments)):
#   a = l_test['comments'][i]
#   b = ' '.join(a)
#   print(b)
#   c = ' '.join(generated_comments[i]) #Get all that comes after the matching string
#   print(c)
#   my_generations.append(c.split(b)[-1])

# l_test['Generated_comments'] = my_generations


In [ ]:
l_test['comments_gen'][0].split('Hit.')

['For customer and watch person date of birth are 1961/12/09 and 1960/12/31 and score is 0.0 with resolution False ',
 '<|endoftext|>']

In [ ]:

#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(l_test)):
  final.append(l_test['comments_gen'][i].split('Hit.')[1].rstrip('<|endoftext|>'))

l_test['Generated_comments'] = final

In [ ]:
final

['',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Date of Birth (DOB)',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Date of Birth (DOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 '',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 '',
 '\n\n0 of 0 found. Print this page.\n\nYosemite Beer Gui',
 'Mismatch Year of Birth (YOB)',
 'Mismatch Year of Birth (YOB)',
 '\n\n1 of 1 customers found this review helpful',
 'Mismatch Year of Birth (YOB)',
 '',
 '\n\n|0         For customer and watch person date of birth are 1988/08/15 and 

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(l_test)):
  reference = l_test['comments'][i]
  candidate = l_test['Generated_comments'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

1.137755747895477e-231

In [ ]:
l_test.to_csv("/content/sample_data/Test_data_comm_gen_4.csv")